<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/Function_Calling_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install openai

In [3]:
import openai
import getpass
import json
apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
openai.api_key = apikey

OpenAIのAPIキーを入力してください··········


In [4]:
def get_current_weather(location, unit):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [5]:
my_functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "unit"],
        },
    }
]

In [6]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
    functions=my_functions,
    function_call="auto",
)

In [7]:
function_name = response["choices"][0]["message"]["function_call"]["name"]

In [8]:
function_name

'get_current_weather'

In [9]:
arguments = response["choices"][0]["message"]["function_call"]["arguments"]

In [10]:
arguments

'{\n  "location": "Boston, MA",\n  "unit": "celsius"\n}'

In [11]:
eval(arguments)

{'location': 'Boston, MA', 'unit': 'celsius'}

In [12]:
function_response = get_current_weather(
    location=eval(arguments)["location"], unit=eval(arguments)["unit"]
)

In [13]:
function_response

'{"location": "Boston, MA", "temperature": "72", "unit": "celsius", "forecast": ["sunny", "windy"]}'

In [14]:
message = response["choices"][0]["message"]

In [16]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What's the weather like in Boston?"},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        },
    ],
)

In [17]:
second_response["choices"][0]["message"]["content"]

'The weather in Boston is currently sunny and windy with a temperature of 72 degrees Celsius.'